## Name: Alhagie Boye
## Title: Search Terms 
## Date: 09/25/2023

# Introduction
The purpose of this lab is to familize with data cleaning, search term analytics, and spellchecking. In this lab I create a list of the most popular search terms (tokens) for a given set of search queries. As people are imperfect, they often misspell search terms, so we are using a spell checking library to remove and interpret misspellings to find the actual most popular terms (and not just the most popular and consistently spelled terms). The search terms come from Direct Supply's DSSI eProcurement system.
Note: panda and numpy are not use in this lab. This lab will be use to compare the Big O analysis of using panda and numpy with the traditional iteration methods.

In [1]:
from spellchecker import SpellChecker
import csv

csv_freq_dict = {}
csv_freq_dict_spellchecked = []

Imports a CSV file and creates a list of the first item of each row. csv: Name of the CSV file
returns A list of the first item of each row of the csv

In [2]:
def import_csv_list_first_col(csv):
    '''
    Imports a csv file and returns a list of the first column of the csv file.
    '''
    temp = []
    csv_raw_data = []
    i = 0
    with open(csv, encoding='utf8') as file:
        for line in file:
            if i == 1000:
                break
            temp.append(line.rstrip('\n').split(','))
            i += 1
    csv_raw_data = [str(row[0]) for row in temp]
    file.closed
    return csv_raw_data


Splits a list of strings into a list of individual words. original_list: List to split
A list of single word strings

In [3]:
def split_tokens(original_list):
    '''
    Splits a list of strings into a list of individual words.
    returns a list of individual words.
    '''
    new_list = []
    for item in original_list:
        new_list.extend(item.split(" "))
    return new_list


This function replaces web spaces with a regular space from a string token

token: String token
returns A string without web spaces

In [4]:
def remove_web_spaces(token):
    token.replace("%20", " ")
    return token

This function removes non-alphabet characters from a string token

token: String token
Returns: A string with only alphabet characters

In [5]:
def remove_non_alphabet(token):
    '''
    Removes all non-alphabet characters from a string.
    e.g. "hello!" -> "hello"
    '''
    fixed_token = ""
    for char in token:
        if char.isalpha() or char == " ":
            fixed_token = fixed_token + char
    return fixed_token

This function creates a frequency dictionary given a string list where the key is a string and the key-value is how many times the string appeared in the list.

input_list: String list
Returns: A frequency dictionary

In [6]:
def list_to_freq_dict(input_list):
    '''
    Takes a list and returns a frequency dictionary of the list.
    returns: dictionary
    '''
    freq_dict = {}
    for i in input_list:
        freq_dict[i] = input_list.count(i)
    return freq_dict

This function creates a sorted frequency list given a frequency dictionary

freq_dict: Frequnecy dictionary
Returns: A 2d list where the first row is frequency and the second row is the string

In [7]:
def sort_freq_dict(freq_dict):
    '''
    Sorts a frequency dictionary from most frequent to least frequent.
    Returns a list of tuples in the format (frequency, word).
    '''
    sorted_list = [(freq_dict[key], key) for key in freq_dict]
    sorted_list.sort()
    sorted_list.reverse()
    return sorted_list

This function creates a spellchecker dictionary where the key is the misspelled word and the key-value is the most likely corrected word

input_list: List of misspelled words
Returns: A spellecheck dictionary

In [8]:
def spellcheck_dict_init(input_list):
    '''
    this function takes a list and returns a dictionary of the input list
    with each value spellchecked.
    '''
    spell = SpellChecker(distance=1)
    spellchecked_dict = {}
    for word in input_list:
        spellchecked_dict[word] = spell.correction(word)
    return spellchecked_dict

This function given a misspelled string token, return the most likely corrected word

token: Misspelled token
Returns: A correctly spelled word

In [9]:
def spellcheck_token(token):
    '''
    This function given a misspelled string token, return the most likely corrected word
    token: Misspelled token  
    Returns: A correctly spelled word
    '''
    fixed_token = csv_spellcheck_dict[token]
    return fixed_token

This function creates a csv file from a frequency list

input_list: Frequency list
returns: NONE

In [10]:
def list_to_csv(input_list):
    '''
    Takes a list of tuples and creates a csv file with the list of tuples.
    '''
    fields = ["Frequency", "Word"]
    with open("Frequency_of_search_terms.csv", "w", newline="") as file:
        write = csv.writer(file)
        write.writerow(fields)
        write.writerows(input_list)
        

# Main

In [11]:
# Import csv to search term list
%time csv_raw = import_csv_list_first_col("searchTerms.csv")

CPU times: total: 0 ns
Wall time: 2.11 ms


In [12]:
%%time
# This section of code filters the data from the csv file by removing non-alphabet characters, 
# replacing web spaces with spaces, and splitting search terms by word

csv_filtered = []
for i in range(len(csv_raw)):
    temp = remove_non_alphabet(remove_web_spaces(csv_raw[i]))
    csv_filtered.append(temp)

CPU times: total: 0 ns
Wall time: 1 ms


In [13]:
%time csv_filtered = split_tokens(csv_filtered)

CPU times: total: 0 ns
Wall time: 0 ns


In [14]:
%%time 
# This section removes all blank search terms. Blank search terms are generated from the above section of code
# due to the removal of non-alphabet characters.
csv_fixed = []
for word in csv_filtered:
    if len(word) != 0:
        csv_fixed.append(word)

CPU times: total: 0 ns
Wall time: 1 ms


In [15]:
# This section of code creates a frequency dictionary of the filtered data.
# A list is also created to sort the frequency dictionary from most frequent to least.
%time csv_freq_dict = list_to_freq_dict(csv_fixed)
%time csv_freq_list = sort_freq_dict(csv_freq_dict)

CPU times: total: 15.6 ms
Wall time: 14.4 ms
CPU times: total: 0 ns
Wall time: 1.07 ms


In [16]:
%%time
# This section of code creates a spellchecked version of the fully filtered search terms list.
# Additionally it creates a frequency dictionary and a sorted list of the most frequent to least
csv_spellcheck_dict = spellcheck_dict_init(csv_fixed)
csv_spellchecked = []
for word in csv_fixed:
    csv_spellchecked.append(spellcheck_token(word))

CPU times: total: 109 ms
Wall time: 104 ms


In [17]:
%time csv_spellcheck_dict = list_to_freq_dict(csv_spellchecked)
%time csv_spellcheck_freq_list = sort_freq_dict(csv_spellcheck_dict)

CPU times: total: 15.6 ms
Wall time: 14 ms
CPU times: total: 0 ns
Wall time: 0 ns


In [18]:
# Creates a csv file of the spellchecked search term frequency list from most frequent to least
%time list_to_csv(csv_spellcheck_freq_list)

CPU times: total: 0 ns
Wall time: 2 ms


# Conclusion

*** The most frequent search tokens for the non-spellchecked data is food items like bacon, milk, chicken, beef. Bacon by far has the most hits with 459, followed by milk at 180 and chicken at 168. This is most likely due to the American breakfast having bacon as one of its components, additionally bacon is the most enjoyed bacon item so this makes sense. Food being the primarily searched items is logical as humans require a large amount of food on a daily basis.***

The spellchecked data in comparison to the original for the most part is the same. Some items saw a slight increase in frequency such as Juice increasing from 131 to 132 when using the spellchecked data. This isn't a surprise as people for the most part can correctly type the name of an item, I hypothesize that the more complex terms will see a much higher increase in hits due to the nature of the spelling.

The spellchecked data in comparison to the original for the most part is the same. Some items saw a slight increase in frequency such as Juice increasing from 131 to 132 when using the spellchecked data. This isn't a surprise as people for the most part can correctly type the name of an item, I hypothesize that the more complex terms will see a much higher increase in hits due to the nature of the spelling.

## Longest-running cell

The longest runing method is the list_to_freq_dict due to the count() method iterating through each item in the list. This is an O(n^2) operation due to each element in the list iterating through the entire list. If the list is 10x bigger it would take 100 times longer and at 100x it would take 10000 times longer.